In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [4]:
import os

cache_dir = "/root/autodl-fs"
os.environ["TRANSFORMERS_CACHE"] = cache_dir

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [6]:
import re, string

In [7]:
model_path = "JiazhenLiu01/llama3-8b-threeturn"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map={"": 0},
    torch_dtype='auto',
    #offload_folder="offload"
    cache_dir=cache_dir
).eval()

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# Read questions from question.txt file and extract them
questions = []
with open("evaluation/evaluation.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        # Extract questions after the question number
        question = line.strip()
        questions.append(question)

In [9]:
for question in questions:
    print(question+"\n")

Could you please share your full name and age with me?

Can you tell me a bit about your family background? Do you have any other family members at home?

What do you typically enjoy doing in your daily life? Do you have any particular interests or hobbies?

I'd like to learn about your school life. Which school do you attend, and how do you feel about it?

Do you have any specific career or vocational goals? What kind of work would you like to do in the future?

Could you tell me about your friends or social circle? Do you have any close friends?

Have you experienced any past mental health issues or therapy? Do you feel these experiences have any impact on your current state?

How would you describe your emotional and mood states in your daily life? Do you sometimes feel low or anxious?

Let's talk about your family relationships. How would you describe your relationships with your family members? Do you feel family dynamics affect your mood?

Lastly, Tom, what do you hope to gain fr

In [10]:
import re

In [11]:
def extract_dialogue(text):
    tom_index = text.find("Tom:")
    if tom_index == -1:
        return None 

    psychologist_index = min(text.find('Psychologist:',tom_index),text.find('Psychologists:',tom_index),text.find('Psychologist',tom_index),text.find('Psychologists',tom_index),
                             text.find('Psychiatrist:',tom_index),text.find('Psychiatrists:',tom_index),text.find('Psychiatrist',tom_index),text.find('Psychiatrists',tom_index),
                             text.find('psychologist:',tom_index),text.find('psychologists:',tom_index),text.find('psychologist',tom_index),text.find('psychologists',tom_index),
                             text.find('psychiatrist:',tom_index),text.find('psychiatrists:',tom_index),text.find('psychiatrist',tom_index),text.find('psychiatrists',tom_index),
                             text.find('\\n',tom_index), text.find('Psychiatry:',tom_index),text.find('\n',tom_index),text.find(r'\b\w*###\w*\b',tom_index),text.find(r'\b\w*Psychologist\w*\b',tom_index),text.find(r'\b\w*psychologist\w*\b',tom_index),text.find(r'\b\w*Psychiatrist\w*\b',tom_index),text.find(r'\b\w*psychiatrist\w*\b',tom_index))
      
                                       

    if psychologist_index != -1:
        dialogue = text[tom_index + len("Tom:"):psychologist_index]
    else:
        dialogue = text[tom_index + len("Tom:"):]

    last_period_index = max(dialogue.rfind('.'),dialogue.rfind('--'),dialogue.rfind('?'),dialogue.rfind('!'),dialogue.rfind(','),dialogue.rfind('-'))

    dialogue = dialogue[:last_period_index+1]
    
    return dialogue.strip() 

In [12]:
# Loop for dialogue inference
with open("evaluation/llama3-threeturn.txt", "a") as output_file:
    for question in questions:
        input_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence. <|eot_id|><|start_header_id|> Psychologist: <|end_header_id|>{question}<|eot_id|><|start_header_id|> Tom: <|end_header_id|>"
        # Encode input text into tokens
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        input_ids = input_ids.to("cuda")
        # Use the trained model for dialogue inference
        output = model.generate(input_ids, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id,max_new_tokens=50)

        # Decode the model output tokens into text
        output_text = tokenizer.decode(output[0], skip_special_tokens=True)

        output_text = output_text.splitlines()[0]
        
        response = extract_dialogue(output_text)

        print("### response:", output_text)
        print("Generated response:", response)

        # Append each answer to falcon_finetuned.txt
        output_file.write("\n"+ response)

### response: systemPentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence.  Psychologist: Could you please share your full name and age with me? Tom: Tom Jones. I'm turning 15 this year.Calibri Psychiatry Clinic - (02) 8312 9000 CalibriPsychiatry.com.au Page | 1 of 8
Generated response: Tom Jones. I'm turning 15 this year.Calibri Psychiatry Clinic - (02) 8312 9000 CalibriPsychiatry.com.
### response: systemPentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence.  Psychologist: Can you tell me a bit about your family background? Do you have any other family members at home? Tom: My mom and dad both live together with their three kids

In [22]:
qa_pairs=[]

In [23]:
# Loop for dialogue inference
with open("llama3-context.txt", "a") as output_file:
    for question in questions:
        input_text = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence."

        for i in range(-4, 0, 1):
            if abs(i) <= len(qa_pairs):
                input_text+=qa_pairs[i]
                print(i)
                
        input_text += f"<|eot_id|><|start_header_id|> Psychologist: <|end_header_id|>{question}<|eot_id|><|start_header_id|> Tom: <|end_header_id|>"
        print(input_text)        
        
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        input_ids = input_ids.to("cuda")
        # Use the trained model for dialogue inference
        output = model.generate(input_ids, repetition_penalty=1.3, pad_token_id=tokenizer.eos_token_id,max_new_tokens=50)
        
        # Decode the model output tokens into text
        output_text = tokenizer.decode(output[0], skip_special_tokens=True)
        
        output_text = output_text.splitlines()[0]
        
        response = extract_dialogue(output_text)
        
        print("Generated response:", response)
        
        # Append each answer to falcon_finetuned.txt
        output_file.write(response+"\n")
        
        qa_pairs.append(f"<|eot_id|><|start_header_id|> Psychologist: <|end_header_id|>{question}")
        qa_pairs.append(f"<|eot_id|><|start_header_id|> Tom: <|end_header_id|>{response}")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English words and respond in a casual tone. Your reply should be no longer than one sentence.<|eot_id|><|start_header_id|> Psychologist: <|end_header_id|>And that does make it a little tricky at the moment because there is the courts involved. Sort of this is the way it looks like.<|eot_id|><|start_header_id|> Tom: <|end_header_id|>
Generated response: Psychologist: And then when things settle down-- or I'm sorry. When things get a bit easier for mom and dad, they can focus on sort of working out what they're going to do about living arrangements and all that sort of stuff.
-2
-1
<|begin_of_text|><|start_header_id|>system<|end_header_id|>Pentending Your name is Tom, and you are now communicating with a psychologist in a therapy session. Respond to the psychologist as Tom using English wor

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.90 GiB. GPU 0 has a total capacty of 23.65 GiB of which 186.06 MiB is free. Process 932046 has 23.46 GiB memory in use. Of the allocated memory 19.85 GiB is allocated by PyTorch, and 3.16 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF